In [64]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [59]:
url = "https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [60]:
disease_urls = {}
for i in 'abcedfghiklmnoprstuvwy':
    div = soup.find('div', class_=f"az_list_indivisual {i}_detail")
    list_items = div.find_all('li')
    for li in list_items:
        anchor_tag = li.find('a')
        disease_name = anchor_tag.text
        link = anchor_tag['href']
        disease_urls[disease_name] = link

In [61]:
disease_urls

{'Abdominal aortic aneurysm': 'https://www.nhsinform.scot/illnesses-and-conditions/heart-and-blood-vessels/conditions/abdominal-aortic-aneurysm/',
 'Acne': 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/acne/',
 'Acute cholecystitis': 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/acute-cholecystitis/',
 'Acute lymphoblastic leukaemia': 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/acute-lymphoblastic-leukaemia/',
 'Acute lymphoblastic leukaemia: Children': 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/acute-lymphoblastic-leukaemia-children/',
 'Acute lymphoblastic leukaemia: Teenagers and young adults': 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/acute-lymphoblastic-leukaemia-teenagers-and-young-adults/',
 'Acute myeloid leukaemia': 'https://www.nhsinform.scot/illnesses-and-condit

In [158]:
disease_dict = {}
for disease,link in tqdm(disease_urls.items()):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    text = ""
    ps = soup.find_all('p')
    uls = soup.find_all('ul')
    for element in soup.find_all():
        if element.name=='p' and ps.index(element) < len(ps)-5 and ps.index(element) >=1:
            text += element.get_text(strip=True)
        elif element.name == 'ul'  and uls.index(element) < len(uls)-5 and uls.index(element) >=3:
            for li in element.find_all('li'):
                text += " " + li.get_text(strip=True)
    text = text.replace('\xa0', ' ').lower()
    
    disease_dict[disease] = text

100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [08:43<00:00,  1.57s/it]


In [159]:
disease_dict

{'Abdominal aortic aneurysm': 'an abdominal aortic aneurysm (aaa) is a swelling (aneurysm) of the aorta – the main blood vessel that leads away from the heart, down through the abdomen to the rest of the body.the abdominal aorta is the largest blood vessel in the body and is usually around 2cm wide – roughly the width of a garden hose. however, it can swell to over 5.5cm – what doctors class as a large aaa.large aneurysms are rare, but can be very serious. if a large aneurysm bursts, it causes huge internal bleeding and is usually fatal.the bulging occurs when the wall of the aorta weakens. although what causes this weakness is unclear, smoking and high blood pressure are thought to increase the risk of an aneurysm.aaas are most common in men aged over 65. a rupture accounts for more than 1 in 50 of all deaths in this group.this is why all men are invited for ascreening testwhen they turn 65. the test involves a simpleultrasound scan, which takes around 10-15 minutes.in most cases, an 

In [160]:
import csv

In [161]:
csv_file = 'disease_info1.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Disease Name', 'Text'])
    
    for disease_name, text in disease_dict.items():
        writer.writerow([disease_name, text])